# Clasificación de pepitas (semillas de calabaza)

## El problema

Imagina que trabajas para una empresa que vende botanas, desde papas fritas hasta semillas de calabaza. Tu empresa se especializa en dos tipos de semillas, puesto que los clientes prefieren esos dos tipos: **ürgüp sivrisi** y **çerçevelik**. Sin embargo, estas dos semillas no son producidas por igual, y una es más común y barata para producir (*sivrisi*) que la otra (*cherchevelik*), y en un estudio de mercado se llegó a la conclusión de que tus clientes son capaces de conocer la diferencia entre ambas semillas, o cuando menos notan que las semillas que están consumiendo no son las que pagaron.

Con el mundial a la vuelta de la esquina, y con el fin de tener el inventario suficiente para poder surtir todos los pedidos, tu empresa ha contactado con diversos proveedores para comprarles sus semillas de calabaza.

Sabiendo que tu empresa se precia de altos estándares de calidad, y con la intención de mantener estos estándares aún con semillas adquiridas externamente, **tu tarea es encontrar una forma de verificar que las semillas recibidas son las que tu empresa está pagando y más importante, vendiendo a tus clientes**.

## La solución

El equipo de control de calidad externo te entrega un archivo CSV en donde cada fila representa una semilla individual y las columnas representan las diversas propiedades de cada una de ellas:

 - area
 - perimeter
 - major_axis_length
 - minor_axis_length
 - convex_area
 - equiv_diameter
 - eccentricity
 - solidity
 - extent
 - roundness
 - aspect_ration
 - compactness
 - class

Tu tarea es entonces crear un modelo capaz de clasificar semillas usando la variable objetivo o dependiente **class** y las variables descriptivas o independientes restantes.

 > ❓ ¿Qué otros métodos podríamos usar?
 
 > ❓ ¿Qué métricas de evaluación podríamos usar?

### Dataset

Este dataset es una versión modificada de el artículo académico *"The use of machine learning methods in classification of pumpkin seeds (Cucurbita pepo L.)"*:

KOKLU, M., SARIGIL, S., & OZBEK, O. (2021). The use of machine learning methods in classification of pumpkin seeds (Cucurbita pepo L.). Genetic Resources and Crop Evolution, 68(7), 2713-2726. Doi: https://doi.org/10.1007/s10722-021-01226-0



## Modelado

In [ ]:
import pandas as pd

seeds_dataset = pd.read_csv("seeds.csv")

In [ ]:
seeds_dataset.head()

### Análisis Exploratorio de Datos (EDA)

Un paso crucial para encontrar las variables más prometedoras a la hora de realizar el modelado.

In [ ]:
from pandas_profiling import ProfileReport

In [ ]:
profile = ProfileReport(seeds_dataset, title="Seed dataset analysis", explorative=True)
profile.to_file("seeds-report.html")

## PCA

In [ ]:
%matplotlib inline
from sklearn.decomposition import PCA
import seaborn as sns

seed_types = seeds_dataset[['class']]
seed_data = seeds_dataset.drop(labels=['area', 'perimeter', 'class'], axis='columns')
pca = PCA(n_components=2)

pca.fit(seed_data)
data = pca.transform(seed_data)

reduced_data = pd.DataFrame(data, columns=['x', 'y'])
reduced_data['class'] = seed_types
reduced_data

sns.scatterplot(x='x', y='y', hue='class', data=reduced_data, alpha=0.5)

## Feature engineering

### Deshazte de las características co-relacionadas

In [ ]:
seeds_dataset_clean = seeds_dataset.drop(labels=['area', 'perimeter'], axis='columns')
seeds_dataset_clean.head()

## Split dataset

In [ ]:
from sklearn.model_selection import train_test_split

rest, test = train_test_split(seeds_dataset_clean, test_size=0.2, shuffle=True) # 20% of 100 = 20
train, val = train_test_split(rest, test_size=0.25, shuffle=True) # 25% of 80 = 20

distributions = [len(train), len(val), len(test)]

print(distributions)
print([dist/len(reduced_data) for dist in distributions])

### Divide features

In [ ]:
train_x = train.drop(labels=['class'], axis='columns')
val_x = val.drop(labels=['class'], axis='columns')
test_x = test.drop(labels=['class'], axis='columns')

train_y = train[['class']].copy()
val_y = val[['class']].copy()
test_y = test[['class']].copy()

print(train_x)
print(train_y)

### Escalado de características (*feature scaling*)

In [ ]:
from sklearn.preprocessing import RobustScaler, StandardScaler, MinMaxScaler, MaxAbsScaler
major_axis_length_scaler = MaxAbsScaler()

In [ ]:
major_axis_length_scaler.fit(train_x[['major_axis_length']])

In [ ]:
major_axis_length_frame = train[['major_axis_length']].copy()
major_axis_length_frame['scaled'] = major_axis_length_scaler.transform(train_x[['major_axis_length']])

In [ ]:
major_axis_length_frame

### Aplicando escalado a múltiples features

In [ ]:
scaler = MaxAbsScaler()

In [ ]:
scaler.fit(train_x)

In [ ]:
train_x_scaled = scaler.transform(train_x)
val_x_scaled = scaler.transform(val_x)
test_x_scaled = scaler.transform(test_x)

train_x_scaled

## Logistic regression

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
lr = LogisticRegression(max_iter=1000, class_weight="balanced")

lr.fit(train_x_scaled, train_y)

## Assess performance

In [ ]:
train_pred = lr.predict(train_x_scaled)  # Para diagnosticar overfitting
val_pred = lr.predict(val_x_scaled)  # Para decidir cambios sobre el modelo

In [ ]:
train_pred

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
training_accuracy = accuracy_score(train_y, train_pred)
validation_accuracy = accuracy_score(val_y, val_pred)

print(f"Training accuracy:   {training_accuracy:0.2%}")
print(f"Validation accuracy: {validation_accuracy:0.2%}")

## Test performance

In [ ]:
test_pred = lr.predict(test_x_scaled)
test_accuracy = accuracy_score(test_y, test_pred)

print(f"Test accuracy:   {test_accuracy:0.2%}")

## Save the artefacts to disk

In [ ]:
import pickle

In [ ]:
with open("scaler.pickle", "wb") as wb:
    pickle.dump(scaler, wb)

In [ ]:
with open("model.pickle", "wb") as wb:
    pickle.dump(lr, wb)

## Reload artifacts from disk

In [ ]:
with open("scaler.pickle", "rb") as rb:
    scaler = pickle.load(rb)

In [ ]:
with open("model.pickle", "rb") as rb:
    model = pickle.load(rb)

## Prueba con tu instancia

In [ ]:
nueva_pepita = [{
    'major_axis_length': 326.1485,
 'minor_axis_length': 220.2388,
 'convex_area': 56831,
 'equiv_diameter': 267.6805,
 'eccentricity': 0.7376,
 'solidity': 0.9902,
 'extent': 0.7453,
 'roundness': 0.8963,
 'aspect_ration': 1.4809,
 'compactness': 0.8207,
}]

nueva_pepita = pd.DataFrame.from_dict(nueva_pepita)
nueva_pepita

In [ ]:
instance_x = scaler.transform(nueva_pepita)

In [ ]:
variant = model.predict(instance_x)
variant_proba = model.predict_proba(instance_x)

In [ ]:
variant[0]